## 3.1 Предобработка изображений

Прогоним нашу выборку через SwinIR - этого будет достаточно, чтобы убрать артефакты/повысить разрешение изображений.
 > Можно сделать обработку и через OpenCV, но там слишком топорно, да и параметры под объектные фильтры придется настраивать - уж лучше поставить +100500 State-of-the-art моделей, они всё сами сделают.

P.S. AUTOMATIC1111 использует встроенный SwinIR, так что попробуем тут SwinIR-Large и затем посмотрим на разницу в генерации тегов.

[![arXiv](https://img.shields.io/badge/arXiv-Paper-<COLOR>.svg)](https://arxiv.org/abs/2108.10257)
[![GitHub Stars](https://img.shields.io/github/stars/JingyunLiang/SwinIR?style=social)](https://github.com/JingyunLiang/SwinIR)


This is a **SwinIR online demo on Real-World Image SR**

P.S. Код почти не менялся, иначе inference ломается



Окружение

In [ ]:
# Clone realESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install -r requirements.txt
!python setup.py develop

# Clone BSRGAN
!git clone https://github.com/cszn/BSRGAN.git

!rm -r SwinIR
# Clone SwinIR
!git clone https://github.com/JingyunLiang/SwinIR.git
!pip install timm

# Download the pre-trained models
#!wget https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN.pth -P experiments/pretrained_models
!wget https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth -P experiments/pretrained_models

Cloning into 'Real-ESRGAN'...
remote: Enumerating objects: 759, done.
remote: Total 759 (delta 0), reused 0 (delta 0), pack-reused 759 (from 1)
Receiving objects: 100% (759/759), 5.39 MiB | 21.97 MiB/s, done.
Resolving deltas: 100% (408/408), done.
/content/Real-ESRGAN
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.3/491.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 10.0 MB/s eta 0:00:00
  Created wheel for basicsr: filename=basicsr-1.4.2-py3-none-any.whl size=214819 sha256=f83a286c5d5ba1fa4d3ffad944c2ea36b9b55ceecb00f8f3e690614c3c41f676
  Stored in directory: /root/.cache/pip/wheels/38/83/99

Переносим наши изображения в определенные директории, чтобы ничего не сломалось

Возьмем **только 5 первых изображений**, больше колаб просто не вывозит на этапе добавления тегов;(

Как и сказано в условии, изображения разнообразные, что будет видно ниже:

In [ ]:
import os
import glob
import shutil

# to be compatible with BSRGAN
!rm -r BSRGAN/testsets/RealSRSet

# /content/Real-ESRGAN/BSRGAN/testsets/RealSRSet
upload_folder = 'BSRGAN/testsets/RealSRSet' #сюда нужно кинуть свои картинки

result_folder = 'results'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.mkdir(upload_folder)
os.mkdir(result_folder)

In [ ]:
!python SwinIR/main_test_swinir.py --task real_sr --model_path experiments/pretrained_models/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth --folder_lq BSRGAN/testsets/RealSRSet --scale 4 --large_model
shutil.move('results/swinir_real_sr_x4_large', 'results/SwinIR_large')
for path in sorted(glob.glob(os.path.join('results/SwinIR_large', '*.png'))):
  os.rename(path, path.replace('SwinIR.png', 'SwinIR_large.png')) # here is a bug in Colab file downloading: no same-name files

# UHD Резы в /content/Real-ESRGAN/results/SwinIR_large

loading model from experiments/pretrained_models/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth
/usr/local/lib/python3.10/dist-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Testing 0 mary                


# **Ссылки**

[Inpaint Anything for SD WebUI ](https://github.com/Uminosachi/sd-webui-inpaint-anything) - отличная вещь для работы с масками

[CLIP Interrogator 2.4](https://colab.research.google.com/github/pharmapsychotic/clip-interrogator/blob/main/clip_interrogator.ipynb#scrollTo=xpPKQR40qvz2) - .ipynb для генерации тегов

[Hotels50K EDA](https://www.kaggle.com/code/michaln/hotels-50k-eda-and-image-download/notebook)

[Stable Diffusion WebUI showcase](https://github.com/AUTOMATIC1111/stable-diffusion-webui-feature-showcase#clip-interrogator) - смотрел тут работу с масками

[Habr](https://habr.com/ru/companies/selectel/articles/712316/) - настройка локального репо AUTOMATIC1111

[BLIP+CLIP | CLIP Interrogator](https://www.kaggle.com/code/leonidkulyk/lb-0-45836-blip-clip-clip-interrogator#--8.-Define-interrogate-function) - статья про более быструю генерацию тегов

[SwinIR](https://github.com/JingyunLiang/SwinIR)

[Tag Clusterization](https://scikit-learn.org/stable/auto_examples/text/plot_document_clustering.html) - Sklearn Documentation

[Tag Clusterization via LDA & Visualization](https://www.kaggle.com/code/panks03/clustering-with-topic-modeling-using-lda/notebook) - красивые картинки на kaggle